In [1]:
import torch
import clip
from PIL import Image
import os

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

frame_folder = "./frames"  
output_file = "frame_vectors.pt"  

image_filenames = [
    os.path.join(frame_folder, fname)
    for fname in sorted(os.listdir(frame_folder))  
    if fname.endswith((".jpg", ".png", ".jpeg"))
]

In [3]:
image_features = []

for image_path in image_filenames:
    try:
        image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
        with torch.no_grad():
            feature = model.encode_image(image)
            feature /= feature.norm(dim=-1, keepdim=True) 
            image_features.append(feature.cpu())
    except Exception as e:
        print(f"이미지 처리 오류: {image_path}, 오류: {e}")

image_features = torch.cat(image_features, dim=0)

torch.save({"filenames": image_filenames, "features": image_features}, output_file)